In [2]:
import numpy as np

from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Flatten
from keras.layers.convolutional import Conv2D
from keras.layers.convolutional import MaxPooling2D
from keras.preprocessing.image import ImageDataGenerator
from keras.layers import Dropout

import keras


# 랜덤시드 고정시키기
np.random.seed(3)

#훈련 데이터와 예측 데이터 생성
train_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
        'datasets/train',
        target_size=(150, 272),
        batch_size= 3,
        color_mode="grayscale",
        class_mode='categorical')

test_datagen = ImageDataGenerator(rescale=1./255)

test_generator = test_datagen.flow_from_directory(
        'datasets/test',
        target_size=(150, 272),
        color_mode="grayscale",
        batch_size=4,
        class_mode='categorical')


# 모델 정의 VGG
model = Sequential()
model.add(Conv2D(128, kernel_size = (3, 3), padding='valid',
                 activation='relu', input_shape=(150, 272, 1)))
model.add(Conv2D(64, (3, 3), padding='valid', activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Conv2D(64, (3, 3), padding='valid', activation='relu'))
model.add(Conv2D(64, (3, 3), padding='valid', activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Flatten())
model.add(Dense(256, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(28, activation='softmax'))

model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

tb_hist = keras.callbacks.TensorBoard(log_dir='./graph', histogram_freq=0, write_graph=True, write_images=True)

#모델 학습시키기
model.fit_generator(
        train_generator,
        steps_per_epoch= 84,
        epochs=10, 
        callbacks = [tb_hist],
        validation_data=test_generator,
        validation_steps=7)


print("-- Evaluate --")
scores = model.evaluate_generator(test_generator, steps=5)
print("%s: %.2f%%" %(model.metrics_names[1], scores[1]*100))

print("-- Predict --")
output = model.predict_generator(test_generator, steps=5)
np.set_printoptions(formatter={'float': lambda x: "{0:0.3f}".format(x)})
print(test_generator.class_indices)

print(test_generator.filenames)


Found 252 images belonging to 28 classes.
Found 28 images belonging to 28 classes.
Epoch 1/10
84/84 [==============================] - 93s 1s/step - loss: 3.3165 - accuracy: 0.0675 - val_loss: 3.2166 - val_accuracy: 0.0357
Epoch 2/10
84/84 [==============================] - 92s 1s/step - loss: 1.9647 - accuracy: 0.4365 - val_loss: 0.0297 - val_accuracy: 0.9286
Epoch 3/10
84/84 [==============================] - 92s 1s/step - loss: 0.4996 - accuracy: 0.8571 - val_loss: 0.0018 - val_accuracy: 0.9286
Epoch 4/10
84/84 [==============================] - 92s 1s/step - loss: 0.2036 - accuracy: 0.9444 - val_loss: 0.0101 - val_accuracy: 0.9643
Epoch 5/10
84/84 [==============================] - 92s 1s/step - loss: 0.1024 - accuracy: 0.9563 - val_loss: 2.8877e-05 - val_accuracy: 1.0000
Epoch 6/10
84/84 [==============================] - 92s 1s/step - loss: 0.1243 - accuracy: 0.9524 - val_loss: 8.5012e-05 - val_accuracy: 0.9643
Epoch 7/10
84/84 [==============================] - 93s 1s/step - los

"\n#데이터 시각화\n\n%matplotlib inline\nimport matplotlib.pyplot as plt\n\nfig, loss_ax = plt.subplots()\n\nacc_ax = loss_ax.twinx()\n\nloss_ax.plot(hist.history['loss'], 'y', label='train loss')\nloss_ax.plot(hist.history['val_loss'], 'r', label='val loss')\n\nacc_ax.plot(hist.history['acc'], 'b', label='train acc')\nacc_ax.plot(hist.history['val_acc'], 'g', label='val acc')\n\nloss_ax.set_xlabel('epoch')\nloss_ax.set_ylabel('loss')\nacc_ax.set_ylabel('accuray')\n\nloss_ax.legend(loc='upper left')\nacc_ax.legend(loc='lower left')\n\nplt.show()\n"